In [1]:
import tensorflow as tf

In [2]:
#Model = tf.keras.Model
#layers = tf.keras.layers.De

In [3]:
import numpy as np
from validation import compute_f1
from keras.models import Model, load_model
from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, \
    Flatten, concatenate
from preprocessing import readfile, createBatches, createMatrices, iterate_minibatches, addCharInformation, padding, createEqualBatches
from keras.utils import plot_model
from keras.initializers import RandomUniform
from keras.optimizers import SGD, Nadam

Using TensorFlow backend.


In [4]:
#data_train = readfile('./datasets/conll2002/esp.train')
#data_test_a = readfile('./datasets/conll2002/esp.testa')
#data_test_b = readfile('./datasets/conll2002/esp.testb')

In [5]:
#trainSentences = readfile("./datasets/corpus/conll2002/esp.train")
#devSentences = readfile("./datasets/corpus/conll2002/esp.testa")
#testSentences = readfile("./datasets/corpus/conll2002/esp.testb")

In [6]:
#charSet = set()

In [25]:
class CNN_BLSTM(object):
    
    def __init__(self, EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, CORPUS):
        
        self.epochs = EPOCHS
        self.dropout = DROPOUT
        self.dropout_recurrent = DROPOUT_RECURRENT
        self.lstm_state_size = LSTM_STATE_SIZE
        self.conv_size = CONV_SIZE
        self.learning_rate = LEARNING_RATE
        self.optimizer = OPTIMIZER
        self.corpus = CORPUS
        
        
    def loadData(self):
        """Load data and add character information"""
        if self.corpus == "conll":
            self.trainSentences = readfile("./datasets/corpus/conll2002/esp.train")
            self.devSentences = readfile("./datasets/corpus/conll2002/esp.testa")
            self.testSentences = readfile("./datasets/corpus/conll2002/esp.testb")
         
        elif self.corpus == "wiki":
            self.trainSentences = readfile('./datasets/corpus/wikiner/wikinerWp3.txt', encoding='latin1')#[0:1000]
            #self.devSentences = readfile("./datasets/corpus/conll2002/esp.testa")
            #self.testSentences = readfile("./datasets/corpus/conll2002/esp.testb")
            dev = readfile('./datasets/corpus/wikiner/wikinerWp2.txt', encoding='latin1')
            self.devSentences = dev[0:400]
            self.testSentences = dev[400:800]
           # del dev
        
        charSet = set()
        for dataset in [self.trainSentences, self.devSentences, self.testSentences]:
             for sentence in dataset:
                #print(sentence)
                for token, label in sentence:
                    for char1 in token:
                        charSet.add(char1)

        self.charList = list(charSet)

    def addCharInfo(self):
        # format: [['EU', ['E', 'U'], 'B-ORG\n'], ...]
        self.trainSentences = addCharInformation(self.trainSentences)
        self.devSentences = addCharInformation(self.devSentences)
        self.testSentences = addCharInformation(self.testSentences)

    def embed(self):
        """Create word- and character-level embeddings"""

        labelSet = set()
        words = {}
        charSet = set()
        # unique words and labels in data  
        for dataset in [self.trainSentences, self.devSentences, self.testSentences]:
            for sentence in dataset:
                for token, char, label in sentence:
                    # token ... token, char ... list of chars, label ... BIO labels   
                    labelSet.add(label)
                    #charSet.add(char)
                    words[token.lower()] = True

        # mapping for labels
        self.label2Idx = {}
        for label in labelSet:
            self.label2Idx[label] = len(self.label2Idx)

        # mapping for token cases
        case2Idx = {'numeric': 0, 'allLower': 1, 'allUpper': 2, 'initialUpper': 3, 'other': 4, 'mainly_numeric': 5,
                    'contains_digit': 6, 'PADDING_TOKEN': 7}
        self.caseEmbeddings = np.identity(len(case2Idx), dtype='float32')  # identity matrix used 

        # read Wikipedia2Vec word embeddings
        word2Idx = {}
        self.wordEmbeddings = []

        if self.corpus == 'conll':
            fEmbeddings = open("./datasets/Embeddings/eswiki_20180420_100d.txt", encoding="utf-8")
            fEmbeddings.readline()
        elif self.corpus == 'wiki':
            fEmbeddings = open("./datasets/Embeddings/eswiki_20180420_100d.txt", encoding="latin1")
            fEmbeddings.readline()

        # loop through each word in embeddings
        for line in fEmbeddings:
            split = line.strip().split(" ")
            word = split[0]  # embedding word entry

            if len(word2Idx) == 0:  # add padding+unknown
                word2Idx["PADDING_TOKEN"] = len(word2Idx)
                vector = np.zeros(len(split) - 1)  # zero vector for 'PADDING' word
                self.wordEmbeddings.append(vector)

                word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
                vector = np.random.uniform(-0.25, 0.25, len(split) - 1)
                self.wordEmbeddings.append(vector)

            if split[0].lower() in words:
                vector = np.array([float(num) for num in split[1:]])
                self.wordEmbeddings.append(vector)  # word embedding vector
                word2Idx[split[0]] = len(word2Idx)  # corresponding word dict

        self.wordEmbeddings = np.array(self.wordEmbeddings)

        # dictionary of all possible characters
        self.char2Idx = {"PADDING": 0, "UNKNOWN": 1}
        #charList=list(charSet)
        for c in self.charList:
        #for c in " 0123456789aábcdeéfghiíjklmnñoópqrstuúüvwxyzAÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚÜVWXYZ.·,-_()[]{}¡!¿?:;#'\"/\\%$`&=*+@^~|<>":
            self.char2Idx[c] = len(self.char2Idx)

        # format: [[wordindices], [caseindices], [padded word indices], [label indices]]
        self.train_set = padding(createMatrices(self.trainSentences, word2Idx, self.label2Idx, case2Idx, self.char2Idx))
        self.dev_set = padding(createMatrices(self.devSentences, word2Idx, self.label2Idx, case2Idx, self.char2Idx))
        self.test_set = padding(createMatrices(self.testSentences, word2Idx, self.label2Idx, case2Idx, self.char2Idx))
        print(self.char2Idx)
        self.idx2Label = {v: k for k, v in self.label2Idx.items()}
        
    def createBatches(self):
        """Create batches"""
        self.train_batch, self.train_batch_len = createBatches(self.train_set)
        self.dev_batch, self.dev_batch_len = createBatches(self.dev_set)
        self.test_batch, self.test_batch_len = createBatches(self.test_set)
        
    def tag_dataset(self, dataset, model):
        """Tag data with numerical values"""
        correctLabels = []
        predLabels = []
        for i, data in enumerate(dataset):
            tokens, casing, char, labels = data
            tokens = np.asarray([tokens])
            casing = np.asarray([casing])
            char = np.asarray([char])
            pred = model.predict([tokens, casing, char], verbose=False)[0]
            pred = pred.argmax(axis=-1)  # Predict the classes
            correctLabels.append(labels)
            predLabels.append(pred)
       #print("CORRECT LABELS:",correctLabels)
        #print("PRED. LABELS:", predLabels)
            
        return predLabels, correctLabels
    
    def buildModel(self):
        """Model layers"""

        # character input
        character_input = Input(shape=(None, 102,), name="Character_input")
        embed_char_out = TimeDistributed(
            Embedding(len(self.char2Idx), 30, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name="Character_embedding")(
            character_input)

        dropout = Dropout(self.dropout)(embed_char_out)

        # CNN
        conv1d_out = TimeDistributed(Conv1D(kernel_size=self.conv_size, filters=30, padding='same', activation='tanh', strides=1), name="Convolution")(dropout)
        maxpool_out = TimeDistributed(MaxPooling1D(102), name="Maxpool")(conv1d_out)
        char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
        char = Dropout(self.dropout)(char)

        # word-level input
        words_input = Input(shape=(None,), dtype='int32', name='words_input')
        words = Embedding(input_dim=self.wordEmbeddings.shape[0], output_dim=self.wordEmbeddings.shape[1], weights=[self.wordEmbeddings],
                          trainable=False)(words_input)

        # case-info input
        casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
        casing = Embedding(output_dim=self.caseEmbeddings.shape[1], input_dim=self.caseEmbeddings.shape[0], weights=[self.caseEmbeddings],
                           trainable=False)(casing_input)

        # concat & BLSTM
        output = concatenate([words, casing, char])
        output = Bidirectional(LSTM(self.lstm_state_size, 
                                    return_sequences=True, 
                                    dropout=self.dropout,                        # on input to each LSTM block
                                    recurrent_dropout=self.dropout_recurrent     # on recurrent input signal
                                   ), name="BLSTM")(output)
        output = TimeDistributed(Dense(len(self.label2Idx), activation='softmax'),name="Softmax_layer")(output)

        # set up model
        self.model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
        
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer=self.optimizer)
        
        self.init_weights = self.model.get_weights()
        
        plot_model(self.model, to_file='model.png')
        
        print("Model built. Saved model.png\n")
        
    def train(self):
        """Default training"""

        self.f1_test_history = []
        self.f1_dev_history = []

        for epoch in range(self.epochs):    
            print("Epoch {}/{}".format(epoch, self.epochs))
            for i,batch in enumerate(iterate_minibatches(self.train_batch,self.train_batch_len)):
                labels, tokens, casing,char = batch       
                self.model.train_on_batch([tokens, casing,char], labels)

            # compute F1 scores
            predLabels, correctLabels = self.tag_dataset(self.test_batch, self.model)
            pre_test, rec_test, f1_test = compute_f1(predLabels, correctLabels, self.idx2Label)
            self.f1_test_history.append(f1_test)
            print("f1 test ", round(f1_test, 4))

            predLabels, correctLabels = self.tag_dataset(self.dev_batch, self.model)
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, self.idx2Label)
            self.f1_dev_history.append(f1_dev)
            print("f1 dev ", round(f1_dev, 4), "\n")
            
        print("Final F1 test score: ", f1_test)
            
        print("Training finished.")
            
        # save model
        self.modelName = "{}_{}_{}_{}_{}_{}_{}_{}".format(self.epochs, 
                                                        self.dropout, 
                                                        self.dropout_recurrent, 
                                                        self.lstm_state_size,
                                                        self.conv_size,
                                                        self.learning_rate,
                                                        self.optimizer.__class__.__name__,
                                                        self.corpus
                                                       )
        
        modelName = self.modelName + ".h5"
        self.model.save(modelName)
        print("Model weights saved.")
        
        self.model.set_weights(self.init_weights)  # clear model
        print("Model weights cleared.")

    def writeToFile(self):
        """Write output to file"""

        # .txt file format
        # [epoch  ]
        # [f1_test]
        # [f1_dev ]
        
        output = np.matrix([[int(i) for i in range(self.epochs)], self.f1_test_history, self.f1_dev_history])

        fileName = self.modelName + ".txt"
        with open(fileName,'wb') as f:
            for line in output:
                np.savetxt(f, line, fmt='%.5f')
                
        print("Model performance written to file.")

    print("Class initialised.")

Class initialised.


In [9]:

"""Set parameters"""

EPOCHS = 5               # paper: 80
DROPOUT = 0.5             # paper: 0.68
DROPOUT_RECURRENT = 0.25  # not specified in paper, 0.25 recommended
LSTM_STATE_SIZE = 275     # paper: 275
CONV_SIZE = 3             # paper: 3
LEARNING_RATE = 0.0105    # paper 0.0105
OPTIMIZER = Nadam()       # paper uses SGD(lr=self.learning_rate), Nadam() recommended
CORPUS = 'wiki'

In [ ]:
cnn_blstm = CNN_BLSTM(EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, CORPUS)
cnn_blstm.loadData()
cnn_blstm.addCharInfo()
cnn_blstm.embed()
#cnn_blstm.createBatches()
#cnn_blstm.buildModel()
#cnn_blstm.train()
#cnn_blstm.writeToFile()

In [ ]:
len(cnn_blstm.trainSentences)

In [ ]:
len(cnn_blstm.devSentences)

In [ ]:
len(cnn_blstm.testSentences)

In [ ]:
#cnn_blstm.testSentences[1]

In [ ]:
cnn_blstm.idx2Label

In [ ]:
#import pandas as pd

In [ ]:
#df = pd.DataFrame.from_dict(cnn_blstm.idx2Label, orient='index').reset_index()

In [ ]:
#df

In [ ]:
#df.columns = ['index', 'label']

In [ ]:
#df.to_pickle('idx2label.pkl', 'gzip')

In [ ]:
cnn_blstm.createBatches()


In [ ]:
len(cnn_blstm.test_batch_len)

In [ ]:
len(cnn_blstm.train_batch)

In [ ]:
len(cnn_blstm.test_batch_len)

In [ ]:
len(cnn_blstm.dev_batch_len)

In [ ]:
cnn_blstm.buildModel()

In [ ]:
cnn_blstm.train()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(cnn_blstm.f1_test_history, label = "F1 test")
plt.plot(cnn_blstm.f1_dev_history, label = "F1 dev")
plt.xlabel("Epochs")
plt.ylabel("F1 score")
plt.legend()
plt.show()

In [ ]:
plt.show()

### Data exploration 

In [26]:
cnn_blstm = CNN_BLSTM(EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, 'wiki')
cnn_blstm.loadData()

category_count = {"B-ORG\n": 0, "I-ORG\n":0, "B-MISC\n": 0, "I-MISC\n":0, "B-LOC\n": 0, "I-LOC\n": 0, "B-PER\n": 0, "I-PER\n": 0, "O\n": 0}
total_count = 0

for sentence in cnn_blstm.trainSentences:
    for word in sentence:
        if word[1] in category_count.keys():
            category_count[word[1]] += 1
            total_count += 1

for category, count in category_count.items():
    print("{}: {}%".format(category.replace("\n", ""), round((count/total_count)*100, 5)))

B-ORG: 0.00049%
I-ORG: 1.18043%
B-MISC: 0.00214%
I-MISC: 2.03657%
B-LOC: 0.00666%
I-LOC: 5.25429%
B-PER: 0.00334%
I-PER: 3.32503%
O: 88.19105%


In [23]:
count = 0
for sentence in cnn_blstm.trainSentences:
    for token in sentence:
        count +=1
        #if token[1] == 'B-LOC\n':
        #    print("B-LOC")

In [24]:
count

264715